In [9]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools
from datetime import datetime

In [10]:
dataset=pd.read_csv("breast-std.csv")

X = dataset.columns[1:]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [11]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [16]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        #gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        mse=mean_squared_error(y, y_predict, squared=False)
        acc=correct/len(y_predict)
        print("Accuracy",correct/len(y_predict))
        print("Errore quadratico medio: ", mse)
        return acc
    
    def accuracy(self, X, y):
        y_predict=self.predict(X)
        correct = np.sum(y_predict == y)
        acc=correct/len(y_predict)
        return acc
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y, decaying_func="linear"):
        n_samples, n_features = X_train.shape
        pos = len(y[y==1])
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)

        if(decaying_func=="linear"):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(decaying_func=="exponential"):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=pos/n_samples
        r_min=1
        self.m=func[0:pos]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[pos:n_samples]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y, decaying_func="linear"):
        
        self.m_func(X_train, y, decaying_func)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
                #Valori di membership?
                #print(y_predict[i]+self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X)) #per l'accuracy
        #return self.project(X) per ritornare le membership

In [17]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
parameters = {'C': C_vals, 'sigma': sigma}
err = []
acc = []
#metto shuffle a False, è richiesto nella classe di indicare la suddivisione nel training set di dati positivi e negativi
skf = StratifiedKFold(n_splits=5, shuffle=False)
print(datetime.now()) 
for train_index, validation_index in skf.split(X, y):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)
    clf = GridSearchCV(model, parameters, cv=5)
    grid_result = clf.fit(X=X_train, y=y_train)
    best_params = clf.best_params_
    best_C = best_params['C']
    best_sigma = best_params['sigma']
    best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)
    print("\nBest model", best_C, best_sigma)
    best_model.fit(X_train, y_train)
    err.append(best_model.score(X_validation, y_validation))
    acc.append(best_model.accuracy(X_validation, y_validation))
print(datetime.now()) 

2020-10-02 15:57:14.625346
     pcost       dcost       gap    pres   dres
 0: -7.1940e+01 -2.2345e+01  2e+03  4e+01  3e-16
 1: -1.5908e+01 -7.9260e+00  1e+02  3e+00  4e-16
 2: -2.3057e+00 -6.7453e+00  7e+00  6e-02  1e-15
 3: -1.9359e+00 -2.4247e+00  5e-01  8e-17  1e-15
 4: -1.9494e+00 -1.9829e+00  3e-02  1e-15  3e-16
 5: -1.9500e+00 -1.9621e+00  1e-02  3e-16  1e-16
 6: -1.9514e+00 -1.9521e+00  6e-04  1e-16  2e-16
 7: -1.9515e+00 -1.9515e+00  1e-05  6e-17  3e-16
 8: -1.9515e+00 -1.9515e+00  1e-07  6e-17  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.1940e+01 -2.2345e+01  2e+03  4e+01  3e-16
 1: -1.5908e+01 -7.9260e+00  1e+02  3e+00  4e-16
 2: -2.3057e+00 -6.7453e+00  7e+00  6e-02  1e-15
 3: -1.9359e+00 -2.4247e+00  5e-01  8e-17  1e-15
 4: -1.9494e+00 -1.9829e+00  3e-02  1e-15  3e-16
 5: -1.9500e+00 -1.9621e+00  1e-02  3e-16  1e-16
 6: -1.9514e+00 -1.9521e+00  6e-04  1e-16  2e-16
 7: -1.9515e+00 -1.9515e+00  1e-05  6e-17  3e-16
 8: -1.9515e+00 -1.9

     pcost       dcost       gap    pres   dres
 0: -6.0384e+01 -2.2826e+01  2e+03  4e+01  3e-16
 1: -1.8052e+01 -8.3694e+00  1e+02  3e+00  5e-16
 2: -2.3684e+00 -6.6739e+00  7e+00  7e-02  2e-15
 3: -1.9501e+00 -2.5033e+00  6e-01  2e-16  2e-15
 4: -1.9646e+00 -2.0158e+00  5e-02  7e-17  4e-16
 5: -1.9647e+00 -2.0060e+00  4e-02  5e-17  4e-16
 6: -1.9672e+00 -1.9754e+00  8e-03  2e-16  3e-16
 7: -1.9680e+00 -1.9694e+00  1e-03  3e-17  3e-16
 8: -1.9682e+00 -1.9683e+00  6e-05  9e-17  4e-16
 9: -1.9682e+00 -1.9682e+00  2e-06  1e-16  4e-16
10: -1.9682e+00 -1.9682e+00  8e-08  1e-16  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.0384e+01 -2.2826e+01  2e+03  4e+01  3e-16
 1: -1.8052e+01 -8.3694e+00  1e+02  3e+00  5e-16
 2: -2.3684e+00 -6.6739e+00  7e+00  7e-02  2e-15
 3: -1.9501e+00 -2.5033e+00  6e-01  2e-16  2e-15
 4: -1.9646e+00 -2.0158e+00  5e-02  7e-17  4e-16
 5: -1.9647e+00 -2.0060e+00  4e-02  5e-17  4e-16
 6: -1.9672e+00 -1.9754e+00  8e-03  2e-16  3e-1

59 out of 85 predictions correct
Accuracy 0.6941176470588235
Errore quadratico medio:  1.1061326375099443
     pcost       dcost       gap    pres   dres
 0: -1.1304e+02 -1.1414e+01  2e+03  5e+01  1e-15
 1: -9.2725e+00 -8.1742e+00  1e+02  2e+00  1e-15
 2: -2.0670e+00 -7.0383e+00  7e+00  5e-02  1e-15
 3: -1.8863e+00 -2.3327e+00  5e-01  9e-04  2e-15
 4: -1.9031e+00 -2.0150e+00  1e-01  2e-04  7e-16
 5: -1.9087e+00 -1.9750e+00  7e-02  7e-05  4e-16
 6: -1.9159e+00 -1.9269e+00  1e-02  9e-06  6e-16
 7: -1.9177e+00 -1.9206e+00  3e-03  1e-06  6e-16
 8: -1.9183e+00 -1.9189e+00  6e-04  1e-07  7e-16
 9: -1.9184e+00 -1.9187e+00  2e-04  3e-08  5e-16
10: -1.9185e+00 -1.9186e+00  7e-05  6e-09  5e-16
11: -1.9185e+00 -1.9185e+00  2e-05  8e-10  6e-16
12: -1.9185e+00 -1.9185e+00  1e-06  5e-17  6e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.1304e+02 -1.1414e+01  2e+03  5e+01  1e-15
 1: -9.2725e+00 -8.1742e+00  1e+02  2e+00  1e-15
 2: -2.0670e+00 -7.0383e+00  7e+00  5e-

56 out of 85 predictions correct
Accuracy 0.6588235294117647
Errore quadratico medio:  1.1682062670406033
     pcost       dcost       gap    pres   dres
 0: -1.3880e+02 -9.0532e+00  2e+03  5e+01  3e-15
 1: -6.6585e+00 -7.6382e+00  6e+01  1e+00  3e-15
 2: -2.1631e+00 -6.4809e+00  5e+00  2e-02  1e-15
 3: -2.0721e+00 -2.2292e+00  2e-01  3e-04  2e-15
 4: -2.0765e+00 -2.1054e+00  3e-02  5e-05  5e-16
 5: -2.0786e+00 -2.0864e+00  8e-03  4e-17  4e-16
 6: -2.0796e+00 -2.0812e+00  2e-03  3e-17  6e-16
 7: -2.0798e+00 -2.0802e+00  4e-04  3e-17  7e-16
 8: -2.0799e+00 -2.0800e+00  1e-04  3e-17  6e-16
 9: -2.0799e+00 -2.0799e+00  3e-05  3e-17  6e-16
10: -2.0799e+00 -2.0799e+00  8e-06  3e-17  6e-16
11: -2.0799e+00 -2.0799e+00  2e-06  3e-17  6e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.3880e+02 -9.0532e+00  2e+03  5e+01  3e-15
 1: -6.6585e+00 -7.6382e+00  6e+01  1e+00  3e-15
 2: -2.1631e+00 -6.4809e+00  5e+00  2e-02  1e-15
 3: -2.0721e+00 -2.2292e+00  2e-01  3e-

59 out of 86 predictions correct
Accuracy 0.686046511627907
Errore quadratico medio:  1.1206310514564426
     pcost       dcost       gap    pres   dres
 0: -1.4521e+02 -7.2326e+00  2e+03  4e+01  2e-15
 1: -3.5749e+00 -7.1957e+00  2e+01  5e-01  2e-15
 2: -2.0680e+00 -4.9638e+00  3e+00  4e-03  1e-15
 3: -2.0593e+00 -2.0931e+00  3e-02  4e-05  6e-16
 4: -2.0600e+00 -2.0603e+00  3e-04  4e-07  6e-16
 5: -2.0600e+00 -2.0600e+00  6e-06  6e-09  6e-16
 6: -2.0600e+00 -2.0600e+00  7e-07  7e-10  6e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4521e+02 -7.2326e+00  2e+03  4e+01  2e-15
 1: -3.5749e+00 -7.1957e+00  2e+01  5e-01  2e-15
 2: -2.0680e+00 -4.9638e+00  3e+00  4e-03  1e-15
 3: -2.0593e+00 -2.0931e+00  3e-02  4e-05  6e-16
 4: -2.0600e+00 -2.0603e+00  3e-04  4e-07  6e-16
 5: -2.0600e+00 -2.0600e+00  6e-06  6e-09  6e-16
 6: -2.0600e+00 -2.0600e+00  7e-07  7e-10  6e-16
Optimal solution found.
64 out of 86 predictions correct
Accuracy 0.7441860465116279
Erro

...

63 out of 86 predictions correct
Accuracy 0.7325581395348837
Errore quadratico medio:  1.034295625950562
     pcost       dcost       gap    pres   dres
 0:  5.1164e+06 -4.2614e+09  4e+09  3e-11  2e-11
 1:  4.4694e+06 -1.2557e+08  1e+08  1e-11  2e-11
 2:  1.9293e+04 -3.3833e+07  3e+07  4e-12  2e-11
 3: -9.4534e+05 -1.9683e+07  2e+07  1e-11  2e-11
 4: -1.3612e+06 -5.0651e+06  4e+06  2e-12  2e-11
 5: -1.5936e+06 -3.0441e+06  1e+06  2e-11  2e-11
 6: -1.7206e+06 -2.7003e+06  1e+06  2e-11  3e-11
 7: -1.8355e+06 -2.2405e+06  4e+05  3e-11  3e-11
 8: -1.8766e+06 -2.0885e+06  2e+05  3e-11  3e-11
 9: -1.9038e+06 -1.9773e+06  7e+04  9e-11  4e-11
10: -1.9186e+06 -1.9301e+06  1e+04  2e-11  4e-11
11: -1.9206e+06 -1.9231e+06  2e+03  4e-11  4e-11
12: -1.9208e+06 -1.9229e+06  2e+03  1e-10  4e-11
13: -1.9211e+06 -1.9217e+06  6e+02  1e-10  4e-11
14: -1.9212e+06 -1.9213e+06  1e+02  7e-11  4e-11
15: -1.9212e+06 -1.9213e+06  5e+01  5e-11  4e-11
16: -1.9212e+06 -1.9213e+06  2e+01  4e-12  4e-11
17: -1.9213e+0

64 out of 85 predictions correct
Accuracy 0.7529411764705882
Errore quadratico medio:  0.9941002434954168
     pcost       dcost       gap    pres   dres
 0: -1.3370e+06 -4.1755e+09  4e+09  2e-11  2e-11
 1: -1.3374e+06 -4.4439e+07  4e+07  6e-11  2e-11
 2: -1.3710e+06 -3.1015e+06  2e+06  4e-11  2e-11
 3: -1.5916e+06 -2.7843e+06  1e+06  5e-11  2e-11
 4: -1.6653e+06 -2.7708e+06  1e+06  3e-11  2e-11
 5: -1.8270e+06 -2.3340e+06  5e+05  3e-11  3e-11
 6: -1.8359e+06 -2.3349e+06  5e+05  9e-12  3e-11
 7: -1.9005e+06 -2.1247e+06  2e+05  7e-12  3e-11
 8: -1.9349e+06 -2.0135e+06  8e+04  1e-11  3e-11
 9: -1.9466e+06 -1.9811e+06  3e+04  3e-11  3e-11
10: -1.9491e+06 -1.9736e+06  2e+04  4e-11  4e-11
11: -1.9535e+06 -1.9592e+06  6e+03  3e-11  4e-11
12: -1.9548e+06 -1.9562e+06  1e+03  6e-11  4e-11
13: -1.9551e+06 -1.9557e+06  7e+02  7e-11  4e-11
14: -1.9552e+06 -1.9554e+06  2e+02  1e-10  4e-11
15: -1.9553e+06 -1.9554e+06  7e+01  5e-11  4e-11
16: -1.9553e+06 -1.9553e+06  2e+01  4e-11  4e-11
17: -1.9553e+

     pcost       dcost       gap    pres   dres
 0: -1.3758e+06 -4.0831e+09  4e+09  4e-12  3e-11
 1: -1.3763e+06 -4.3741e+07  4e+07  1e-11  3e-11
 2: -1.4165e+06 -3.3217e+06  2e+06  6e-11  3e-11
 3: -1.5993e+06 -3.0989e+06  1e+06  1e-11  3e-11
 4: -1.6687e+06 -3.1018e+06  1e+06  4e-11  2e-11
 5: -1.8755e+06 -2.5859e+06  7e+05  3e-11  3e-11
 6: -1.9233e+06 -2.5353e+06  6e+05  7e-12  3e-11
 7: -2.0314e+06 -2.1760e+06  1e+05  7e-12  3e-11
 8: -2.0563e+06 -2.1093e+06  5e+04  2e-11  3e-11
 9: -2.0605e+06 -2.0968e+06  4e+04  3e-11  3e-11
10: -2.0676e+06 -2.0742e+06  7e+03  5e-11  4e-11
11: -2.0691e+06 -2.0711e+06  2e+03  5e-11  4e-11
12: -2.0695e+06 -2.0702e+06  7e+02  6e-11  4e-11
13: -2.0696e+06 -2.0699e+06  2e+02  8e-11  4e-11
14: -2.0697e+06 -2.0698e+06  1e+02  3e-11  4e-11
15: -2.0697e+06 -2.0697e+06  5e+01  4e-11  4e-11
16: -2.0697e+06 -2.0697e+06  1e+01  2e-11  4e-11
17: -2.0697e+06 -2.0697e+06  3e+00  2e-11  5e-11
18: -2.0697e+06 -2.0697e+06  9e-01  2e-11  4e-11
Optimal solution foun

56 out of 85 predictions correct
Accuracy 0.6588235294117647
Errore quadratico medio:  1.1682062670406033
     pcost       dcost       gap    pres   dres
 0: -1.4469e+06 -4.0821e+09  4e+09  4e-12  3e-11
 1: -1.4474e+06 -4.2889e+07  4e+07  1e-11  3e-11
 2: -1.4891e+06 -2.4975e+06  1e+06  2e-11  3e-11
 3: -2.0725e+06 -2.1048e+06  3e+04  1e-11  3e-11
 4: -2.0768e+06 -2.0912e+06  1e+04  7e-12  3e-11
 5: -2.0792e+06 -2.0824e+06  3e+03  4e-11  3e-11
 6: -2.0796e+06 -2.0808e+06  1e+03  5e-11  3e-11
 7: -2.0798e+06 -2.0802e+06  4e+02  4e-11  4e-11
 8: -2.0799e+06 -2.0800e+06  1e+02  5e-11  4e-11
 9: -2.0799e+06 -2.0799e+06  2e+01  3e-11  4e-11
10: -2.0799e+06 -2.0799e+06  4e+00  4e-11  4e-11
11: -2.0799e+06 -2.0799e+06  1e+00  4e-11  4e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.4469e+06 -4.0821e+09  4e+09  4e-12  3e-11
 1: -1.4474e+06 -4.2889e+07  4e+07  1e-11  3e-11
 2: -1.4891e+06 -2.4975e+06  1e+06  2e-11  3e-11
 3: -2.0725e+06 -2.1048e+06  3e+04  1e-

RuntimeError: Cannot clone object HYP_SVM(C=1, sigma=0.09), as the constructor either does not set or modifies parameter C

In [ ]:
print(np.mean(err), np.mean(acc))

In [ ]:
print(err, acc)